In [1]:
# For testing the performance of Tensor Random Projection on Recommendation System
from tr_functions import *
import timeit
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# MovieLens Latest Datasets: https://grouplens.org/datasets/movielens/
# users: 610, movies: 9743
ratings = pd.read_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\ml-edu\ratings.csv')
# time_stamp = np.array(ratings['timestamp'])
# time_gap = (time_stamp.max()-time_stamp.min())/365/24/3600
#print('The time gap of the data is', time_gap, 'years.')
#print(ratings.dtypes)
ratings.userId = ratings.userId.astype('category')
ratings.movieId = ratings.movieId.astype('category')
ratings.timestamp = ratings.timestamp.astype('category')
#print(ratings.dtypes)

ratings['userIndex'] = ratings.userId.cat.codes
ratings['movieIndex'] = ratings.movieId.cat.codes
ratings['timeIndex'] = ratings.timestamp.cat.codes
ratings.sort_values("timeIndex", inplace=True, ascending=True)
#print(ratings.shape)
#ratings.head()
#ratings['time']

#print('Number of users:', ratings['userIndex'].max())
#print('Number of movies:', ratings['movieIndex'].max())
#print('Number of timelines:', ratings['timeIndex'].max())
rating_tensor = np.zeros((ratings['userIndex'].max()+1, ratings['movieIndex'].max()+1, 23))
#print('Divide the ratings in 22 years')
#print('The shape of the tensor data is', rating_tensor.shape)
seq = ratings['timeIndex'].max()//22
k = 1
for i in range(100836):
    if ratings['timeIndex'][i]//seq ==k:
        k += 1
        # print(ratings['timeIndex'][i], k)
    rating_tensor[ratings['userIndex'][i], ratings['movieIndex'][i], k-1] = ratings['rating'][i]
    

data = np.copy(rating_tensor)
data[data > 0] = 1
print('The training data is of size', rating_tensor.shape, 'and contains', int(data.sum()), 'ratings.')
#print('The tensor structure is user * movies * year of size 610 * 9724 * 23.')

The training data is of size (610, 9724, 23) and contains 100836 ratings.


In [3]:
# Train test split to test the generalization on test data
X_train, X_test, y_train, y_test = train_test_split(ratings[['userIndex', 'movieIndex', 'timeIndex']].values, 
                                                    ratings['rating'].values, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# formulate training tensor
rating_tensor_train = np.zeros((ratings['userIndex'].max()+1, ratings['movieIndex'].max()+1, 23))
seq = ratings['timeIndex'].max()//22
for i in range(X_train.shape[0]):
    rating_tensor_train[X_train[i, 0], X_train[i, 1], X_train[i, 2]//seq] = y_train[i]
    
# formulate test tensor
rating_tensor_test = np.zeros((ratings['userIndex'].max()+1, ratings['movieIndex'].max()+1, 23))

seq = ratings['timeIndex'].max()//22
for i in range(X_test.shape[0]):
    rating_tensor_test[X_test[i, 0], X_test[i, 1], X_test[i, 2]//seq] = y_test[i]
    
# formulate training matrix
rating_matrix_train = np.zeros((ratings['userIndex'].max()+1, ratings['movieIndex'].max()+1))
for i in range(X_train.shape[0]):
    rating_matrix_train[X_train[i, 0], X_train[i, 1]] = y_train[i]
    
# formulate test matrix
rating_matrix_test = np.zeros((ratings['userIndex'].max()+1, ratings['movieIndex'].max()+1))
for i in range(X_test.shape[0]):
    rating_matrix_test[X_test[i, 0], X_test[i, 1]] = y_test[i]


data1 = np.copy(rating_tensor_train)
data1[data1 > 0] = 1
data2 = np.copy(rating_tensor_test)
data2[data2 > 0] = 1
print('The training/test tensor data is of size', rating_tensor_train.shape, 'and contains', int(data1.sum()),'/',int(data2.sum()), 'ratings.')
print('The training/test matrix data is of size', rating_matrix_train.shape, 'and contains', int(data1.sum()),'/',int(data2.sum()), 'ratings.')

(80668, 3) (20168, 3) (80668,) (20168,)
The training/test tensor data is of size (610, 9724, 23) and contains 80668 / 20168 ratings.
The training/test matrix data is of size (610, 9724) and contains 80668 / 20168 ratings.


In [4]:
# input mean
user_ratings_mean = rating_tensor_train[rating_tensor_train>0].mean()
rating_tensor_train_addmean = rating_tensor_train.copy()
rating_tensor_train_addmean[rating_tensor_train_addmean==0] = user_ratings_mean

rating_matrix_train_addmean = rating_matrix_train.copy()
rating_matrix_train_addmean[rating_matrix_train_addmean==0] = user_ratings_mean

In [5]:
# Train on the training data


sampled_tensor = rating_tensor_train_addmean[0:500, 0:1000, :] # downsample to 500*1000*20 for fast calculation
sampled_matrix = rating_matrix_train_addmean[0:500, 0:1000] # downsample to 500*1000 for fast calculation

# TRP-TR-ALS
projection_size = [20, 20, 10]
tr_rank = [2, 2, 2]


tensor_p, Q = tensor_projection(sampled_tensor, projection_size, 2,  0)
start1 = timeit.default_timer()
cores_projection = TR_ALS(tensor_p, tr_rank, maxiter=10)
# print('Tensor decomposition completed.')
cores1 = tensor_back_projection(cores_projection, Q)
tensor1 = cores2tensor_new(cores1)
# print('Rating prediction completed.')
stop1 = timeit.default_timer()

# prediction1 = tensor1 + user_ratings_mean

# TRALS
start2 = timeit.default_timer()
tensor2 = cores2tensor_new(TR_ALS(sampled_tensor, tr_rank, maxiter=10))
stop2 = timeit.default_timer()

# prediction2 = tensor2 + user_ratings_mean

# TR-ALS matrix
tr_rank_m = [2, 2]
start3 = timeit.default_timer()
tensor3 = cores2tensor_new(TR_ALS(sampled_matrix, tr_rank, maxiter=10))
stop3 = timeit.default_timer()



Converging TR-ALS
..........Finished!
Converging TR-ALS
..........Finished!
Converging TR-ALS
..........Finished!


In [6]:
# Evaluation

sampled_tensor_test = rating_tensor_test[0:500, 0:1000, :]
index = np.int64(sampled_tensor_test>0) # a binary tesor marking the exsiting ratings as 1
num = index.sum() # number of ratings

sampled_matrix_test = rating_matrix_test[0:500, 0:1000]
index_m = np.int64(sampled_matrix_test>0) # a binary tesor marking the exsiting ratings as 1


prediction_tensor1 = tensor1 * index
prediction_tensor2 = tensor2 * index
prediction_tensor3 = tensor3 * index_m

# Evaluation on test dataset
MAE1 = evaluation_MAE(sampled_tensor_test, prediction_tensor1*index)*tensor1.size/num
MAE2 = evaluation_MAE(sampled_tensor_test, prediction_tensor2*index)*tensor2.size/num
MAE3 = evaluation_MAE(sampled_matrix_test, prediction_tensor3*index_m)*tensor3.size/num

RMSE1 = evaluation_RMSE(sampled_tensor_test, prediction_tensor1)*np.sqrt(tensor1.size/num)
RMSE2 = evaluation_RMSE(sampled_tensor_test, prediction_tensor2)*np.sqrt(tensor2.size/num)
RMSE3 = evaluation_RMSE(sampled_matrix_test, prediction_tensor3)*np.sqrt(tensor3.size/num)


print('Time spend of TRP-TR-ALS is %.2fs, time spend of TR-ALS is %.2fs, time spend of TR-ALS matrix is %.2fs' %(stop1 - start1, stop2 - start2, stop3-start3))
print('MAE of TRP-TR-ALS is %.4f, MAE of TR-ALS is %.4f, MAE of TR-ALS matrix is %.4f,' %(MAE1, MAE2, MAE3))
print('RMSE of TRP-TR-ALS is %.4f, RMSE of TR-ALS is %.4f, RMSE of TR-ALS matrix is %.4f,' %(RMSE1, RMSE2, RMSE3))


Time spend of TRP-TR-ALS is 0.34s, time spend of TR-ALS is 9.12s, time spend of TR-ALS matrix is 0.06s
MAE of TRP-TR-ALS is 0.8394, MAE of TR-ALS is 0.8265, MAE of TR-ALS matrix is 0.7590,
RMSE of TRP-TR-ALS is 1.0215, RMSE of TR-ALS is 1.0104, RMSE of TR-ALS matrix is 0.9417,


In [7]:
# precision and recall
sampled_tensor_rec = sampled_tensor_test.copy()
sampled_tensor_rec[sampled_tensor_test>=3.5] = 1
sampled_tensor_rec[sampled_tensor_test <3.5 ] = 0
sampled_rec_real = sampled_tensor_rec[index!=0]

def rating(prediction, user, item):
    a = 1
    b = 0
    rating = prediction[user-1, item-1, :]
    if rating.max() >=3.5:
        return a
    else: 
        return b
tensor1_rec = prediction_tensor1.copy()
tensor1_rec[prediction_tensor1>=3.5] = 1
tensor1_rec[prediction_tensor1<3.5] = 0
sampled_rec_1 = tensor1_rec[index!=0]
print(sampled_rec_1)
print(sampled_rec_real)
num_wrong = np.sum(np.abs(sampled_rec_1 - sampled_rec_real))
num_correct = sampled_rec_1.size - num_wrong

precision = num_correct/sampled_rec_1.size
print(precision)

[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 0. 1.]
0.7032191069574247
